In [8]:
### NEW: install and import sqlalchemy
!pip install sqlalchemy
import sqlalchemy
sqlalchemy.__version__

'2.0.43'

In [6]:
### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')


### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [12]:
#Exercise 1: Open the Database

##open the database using sqlalchemy module interface.
import sqlalchemy
from sqlalchemy import create_engine

#create an engine object in a variable named engine
engine = create_engine("sqlite:///chinook.db")

# Connect to the database
conn = engine.connect()
print("Database opened successfully!")

### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

Database opened successfully!


In [13]:
#Exercise 2: Table Names

##print out all the table names

from sqlalchemy import MetaData, Table

# Create a MetaData instance
metadata = MetaData()

# Reflect the tables from the database
metadata.reflect(bind=engine)

print(metadata.tables.keys())


dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


In [15]:
#Exercise 3: Tracks
##print out the first three tracks in the tracks table

from sqlalchemy import select
trackstable = metadata.tables['tracks']
query = select(trackstable).limit(3)
result = conn.execute(query)

rows = result

for row in rows:
  print(row)

(1, 'For Those About To Rock (We Salute You)', 1, 1, 1, 'Angus Young, Malcolm Young, Brian Johnson', 343719, 11170334, Decimal('0.99'))
(2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, Decimal('0.99'))
(3, 'Fast As a Shark', 3, 2, 1, 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 230619, 3990994, Decimal('0.99'))


In [30]:
#Exercise 4 : Albums from Tracks
## Print out the track name and albums title of the first 20 tracks in the tracks table
for column in trackstable.columns:
  print(f"{column.name}: {column.type}") #confirm the column names

albumstable = metadata.tables['albums']
for column in albumstable.columns:
  print(f"{column.name}: {column.type}") # confirm the column names

track_album_query = (select(trackstable.c.Name, albumstable.c.Title)
.join(albumstable, trackstable.c.AlbumId == albumstable.c.AlbumId)
    .limit(20)
)

track_album_result = conn.execute(track_album_query).fetchall()

# print results
for row in track_album_result:
    print(row)

display_results(track_album_query)

TrackId: INTEGER
Name: NVARCHAR(200)
AlbumId: INTEGER
MediaTypeId: INTEGER
GenreId: INTEGER
Composer: NVARCHAR(220)
Milliseconds: INTEGER
Bytes: INTEGER
UnitPrice: NUMERIC(10, 2)
AlbumId: INTEGER
Title: NVARCHAR(160)
ArtistId: INTEGER
('For Those About To Rock (We Salute You)', 'For Those About To Rock We Salute You')
('Put The Finger On You', 'For Those About To Rock We Salute You')
("Let's Get It Up", 'For Those About To Rock We Salute You')
('Inject The Venom', 'For Those About To Rock We Salute You')
('Snowballed', 'For Those About To Rock We Salute You')
('Evil Walks', 'For Those About To Rock We Salute You')
('C.O.D.', 'For Those About To Rock We Salute You')
('Breaking The Rules', 'For Those About To Rock We Salute You')
('Night Of The Long Knives', 'For Those About To Rock We Salute You')
('Spellbound', 'For Those About To Rock We Salute You')
('Balls to the Wall', 'Balls to the Wall')
('Fast As a Shark', 'Restless and Wild')
('Restless and Wild', 'Restless and Wild')
('Princes

,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



In [38]:
#Exercise5: Tracks sold

invoiceitemstable = metadata.tables['invoice_items']
for column in invoiceitemstable.columns:
  print(f"{column.name}: {column.type}") #confirm the column names

invoicequery = select(trackstable.c.Name, invoiceitemstable.c.Quantity).select_from(invoiceitemstable.join(trackstable, trackstable.c.TrackId == invoiceitemstable.c.TrackId)).limit(10)## print out the first 10 track sales with the names of the track sold, and the quantity sold




invoiceresult = conn.execute(invoicequery).fetchall()

# print results
for row in invoiceresult:
    print(row)

display_results(invoicequery)


InvoiceLineId: INTEGER
InvoiceId: INTEGER
TrackId: INTEGER
UnitPrice: NUMERIC(10, 2)
Quantity: INTEGER
('Balls to the Wall', 1)
('Restless and Wild', 1)
('Put The Finger On You', 1)
('Inject The Venom', 1)
('Evil Walks', 1)
('Breaking The Rules', 1)
('Dog Eat Dog', 1)
('Overdose', 1)
('Love In An Elevator', 1)
("Janie's Got A Gun", 1)


,Name,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



In [48]:
#Exercise 6 : Top tracks sold

from sqlalchemy import func


query = (
    select(trackstable.c.Name,
           func.sum(invoiceitemstable.c.Quantity).label("TotalSold"))
    .join(trackstable, trackstable.c.TrackId == invoiceitemstable.c.TrackId)
    .group_by(invoiceitemstable.c.TrackId)
    .order_by(func.sum(invoiceitemstable.c.Quantity).desc())
    .limit(10)
) #print the names of top 10 tracks sold, and how many they times they were sold

result = conn.execute(query).fetchall()
display_results(query)



,Name,TotalSold
0,Balls to the Wall,2
1,Inject The Venom,2
2,Snowballed,2
3,Overdose,2
4,Deuces Are Wild,2
5,Not The Doctor,2
6,Por Causa De Você,2
7,Welcome Home (Sanitarium),2
8,Snowblind,2
9,Cornucopia,2



SELECT tracks."Name", sum(invoice_items."Quantity") AS "TotalSold" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId" GROUP BY invoice_items."TrackId" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT :param_1



In [72]:
#Exercise 7 - Who are the top 10 highest selling artists?
artiststable = metadata.tables['artists']
albumstable = metadata.tables['albums']

query = select(artiststable.c.Name.label("Top Selling Artists"),
        func.sum(invoiceitemstable.c.Quantity).label("TotalSold")).join(trackstable,trackstable.c.TrackId == invoiceitemstable.c.TrackId).join(albumstable,albumstable.c.AlbumId == trackstable.c.AlbumId).join(artiststable, artiststable.c.ArtistId == albumstable.c.ArtistId).group_by(artiststable.c.Name).order_by(func.sum(invoiceitemstable.c.Quantity).desc()).limit(10)


result = conn.execute(query)

display_results(query)





,Top Selling Artists,TotalSold
0,Iron Maiden,140
1,U2,107
2,Metallica,91
3,Led Zeppelin,87
4,Os Paralamas Do Sucesso,45
5,Deep Purple,44
6,Faith No More,42
7,Lost,41
8,Eric Clapton,40
9,R.E.M.,39



SELECT artists."Name" AS "Top Selling Artists", sum(invoice_items."Quantity") AS "TotalSold" 
FROM invoice_items JOIN tracks ON tracks."TrackId" = invoice_items."TrackId" JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN artists ON artists."ArtistId" = albums."ArtistId" GROUP BY artists."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT :param_1



In [ ]:




    dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])



    .group_by(invoice_items_table.c.TrackId)

query = (
    select(trackstable.c.Name,
           func.sum(invoiceitemstable.c.Quantity).label("TotalSold"))
    .join(trackstable, trackstable.c.TrackId == invoiceitemstable.c.TrackId)
    .group_by(invoiceitemstable.c.TrackId)
    .order_by(func.sum(invoiceitemstable.c.Quantity).desc())
    .limit(10)
) #print the names of top 10 tracks sold, and how many they times they were sold

